In [210]:
import os
import json
import pandas as pd
from datetime import datetime, timezone
import requests
import re

In [211]:
%pwd

'C:\\Users\\pc\\Desktop\\GitHub repos\\End-to-end-ML-project-with-MLflow'

In [212]:
os.chdir("C:/Users/pc/Desktop/GitHub repos/End-to-end-ML-project-with-MLflow")

In [213]:
%pwd

'C:\\Users\\pc\\Desktop\\GitHub repos\\End-to-end-ML-project-with-MLflow'

In [214]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [215]:
from src.mlProject.constants.__init__ import *
from src.mlProject.utils.common import read_yaml, create_dirs

In [216]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dirs([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config

In [217]:
import os
import urllib.request as request
import patoolib
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [218]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_archive_file(self):
        """
        Extracts the archive file (zip or rar) into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        try:
            patoolib.extract_archive(self.config.local_data_file, outdir=unzip_path)
            logger.info(f"Successfully extracted archive to {unzip_path}")
        except Exception as e:
            logger.error(f"Error extracting archive: {str(e)}")
            raise e

In [220]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_archive_file()
except Exception as e:
    raise e

[2025-05-19 22:22:24,332: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-19 22:22:24,335: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-19 22:22:24,337: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-19 22:22:24,339: INFO: common: created directory at: artifacts]
[2025-05-19 22:22:24,340: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-19 22:22:28,873: INFO: 1833220833: artifacts/data_ingestion/velib_data.rar download! with following info: 
Connection: close
Content-Length: 41219934
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/octet-stream
ETag: "dd9b7106cfcfde11a23c3e79aa1dba5931bb5e51b19b918163c0effa986f420d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A9BC:3C7ED1:4382C81:4A1B585:682B9207
Accept-Ranges:

INFO patool: Extracting artifacts/data_ingestion/velib_data.rar ...


[2025-05-19 22:22:28,875: INFO: log: Extracting artifacts/data_ingestion/velib_data.rar ...]


INFO patool: running "C:\Program Files\WinRAR\rar.EXE" x -kb -or -- "C:\Users\pc\Desktop\GitHub repos\End-to-end-ML-project-with-MLflow\artifacts\data_ingestion\velib_data.rar"


[2025-05-19 22:22:28,877: INFO: log: running "C:\Program Files\WinRAR\rar.EXE" x -kb -or -- "C:\Users\pc\Desktop\GitHub repos\End-to-end-ML-project-with-MLflow\artifacts\data_ingestion\velib_data.rar"]


INFO patool: ... artifacts/data_ingestion/velib_data.rar extracted to `artifacts/data_ingestion'.


[2025-05-19 22:22:29,793: INFO: log: ... artifacts/data_ingestion/velib_data.rar extracted to `artifacts/data_ingestion'.]
[2025-05-19 22:22:29,795: INFO: 1833220833: Successfully extracted archive to artifacts/data_ingestion]
